<h3>Import Libraries

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read Data
path = '/content/drive/MyDrive/NLP/Sentimental Analysis Project/Amazon_Customer_Reviews.csv'
data = pd.read_csv(path)
data.drop('Unnamed: 0',axis=1,inplace=True)
data.head()


,Ratings,Reviews
0,Good,"I use Amazon.com often, and 99% of the time, ..."
1,Good,With Amazon you can leisurely shop in the com...
2,Good,I am very happy with all my purchases since I...
3,Good,"I absolutely love Amazon. Their selections, pr..."
4,Good,We have shopped Amazon for years and always fi...


<h3>Pre-Processing

In [7]:
#Data cleaning and preprocessing
import re
from nltk.stem import WordNetLemmatizer
wordnet = WordNetLemmatizer()

# Function for preprocessing
def preprocessing(data):    
    
    reviews =  []
    for i in range(0, len(data)):
        # Replacing values in rows
        review = data['Reviews'][i]
        review = review.replace('$','dollars')
        review = review.lower()
        reviews.append(review)
    
    return reviews
        
reviews_1 = preprocessing(data)

In [8]:
df = pd.DataFrame(reviews_1)
df.head()

,0
0,"i use amazon.com often, and 99% of the time, ..."
1,with amazon you can leisurely shop in the com...
2,i am very happy with all my purchases since i...
3,"i absolutely love amazon. their selections, pr..."
4,we have shopped amazon for years and always fi...


In [9]:
df.columns=['reviews']

In [10]:
df.head()

,reviews
0,"i use amazon.com often, and 99% of the time, ..."
1,with amazon you can leisurely shop in the com...
2,i am very happy with all my purchases since i...
3,"i absolutely love amazon. their selections, pr..."
4,we have shopped amazon for years and always fi...


<h3> Word to vec Representation

In [11]:
path_vec = '/content/drive/MyDrive/NLP/Sentimental Analysis Project/wiki-news-300d-1M.vec'

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open(path_vec)):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [ ]:
embeddings_index

In [ ]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(df['reviews'])
word_index = token.word_index

In [ ]:
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.0281    ,  0.2958    ,  0.0666    , ..., -0.0482    ,
         0.14300001,  0.0724    ],
       [ 0.0171    ,  0.0596    , -0.0121    , ...,  0.0741    ,
         0.004     , -0.0612    ],
       ...,
       [-0.0885    , -0.1832    , -0.084     , ...,  0.15350001,
        -0.0632    , -0.0021    ],
       [ 0.0506    , -0.17900001, -0.20730001, ...,  0.1452    ,
        -0.0499    ,  0.1106    ],
       [ 0.0481    , -0.02      , -0.0963    , ..., -0.0867    ,
         0.0005    , -0.35960001]])

<h3>Train-Test Split and Dummifying output variable 

In [ ]:
#dummyfying output variable
y=pd.get_dummies(data['Ratings'],drop_first=True)

# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['reviews'], y)

In [ ]:
len(max(df['reviews']))

165

In [ ]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=165)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=165)

In [ ]:
train_seq_x

array([[   0,    0,    0, ...,    8, 4545,  207],
       [1684,   23,  909, ...,   78,   76,   56],
       [   0,    0,    0, ...,    3,   25, 1061],
       ...,
       [   0,    0,    0, ...,   81,   14,  630],
       [   0,    0,    0, ...,  151,   83,   21],
       [   2,  466,   73, ...,   78,   76,   56]], dtype=int32)

<h3>LSTM

In [ ]:
## Creating model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

model=Sequential()
model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix],input_length=165 ,trainable=False))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 165, 300)          5350800   
_________________________________________________________________
dropout_1 (Dropout)          (None, 165, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 5,511,301
Trainable params: 160,501
Non-trainable params: 5,350,800
_________________________________________________________________
None


In [ ]:
model.fit(train_seq_x,train_y,validation_data=(valid_seq_x,valid_y),epochs=10,batch_size=64)

Epoch 1/10
78/78 [==============================] - 30s 361ms/step - loss: 0.5573 - accuracy: 0.6945 - val_loss: 0.2863 - val_accuracy: 0.9022
Epoch 2/10
78/78 [==============================] - 28s 354ms/step - loss: 0.2476 - accuracy: 0.9147 - val_loss: 0.2451 - val_accuracy: 0.9155
Epoch 3/10
78/78 [==============================] - 28s 356ms/step - loss: 0.2442 - accuracy: 0.9159 - val_loss: 0.2241 - val_accuracy: 0.9269
Epoch 4/10
78/78 [==============================] - 28s 356ms/step - loss: 0.2075 - accuracy: 0.9323 - val_loss: 0.2106 - val_accuracy: 0.9312
Epoch 5/10
78/78 [==============================] - 28s 356ms/step - loss: 0.1879 - accuracy: 0.9378 - val_loss: 0.2051 - val_accuracy: 0.9342
Epoch 6/10
78/78 [==============================] - 28s 355ms/step - loss: 0.1709 - accuracy: 0.9482 - val_loss: 0.2080 - val_accuracy: 0.9318
Epoch 7/10
78/78 [==============================] - 28s 356ms/step - loss: 0.1658 - accuracy: 0.9491 - val_loss: 0.2233 - val_accuracy: 0.9287

<h3>Gated Recurrent Network

In [ ]:
## Creating model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense

model=Sequential()
model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix],input_length=165 ,trainable=False))
model.add(Dropout(0.3))
model.add(GRU(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 165, 300)          5350800   
_________________________________________________________________
dropout_2 (Dropout)          (None, 165, 300)          0         
_________________________________________________________________
gru (GRU)                    (None, 100)               120600    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 5,471,501
Trainable params: 120,701
Non-trainable params: 5,350,800
_________________________________________________________________
None


In [ ]:
model.fit(train_seq_x,train_y,validation_data=(valid_seq_x,valid_y),epochs=10,batch_size=64)

Epoch 1/10
78/78 [==============================] - 25s 301ms/step - loss: 0.5639 - accuracy: 0.6830 - val_loss: 0.4674 - val_accuracy: 0.7645
Epoch 2/10
78/78 [==============================] - 23s 299ms/step - loss: 0.4328 - accuracy: 0.8006 - val_loss: 0.3039 - val_accuracy: 0.8853
Epoch 3/10
78/78 [==============================] - 23s 300ms/step - loss: 0.2476 - accuracy: 0.9162 - val_loss: 0.2132 - val_accuracy: 0.9275
Epoch 4/10
78/78 [==============================] - 23s 298ms/step - loss: 0.1886 - accuracy: 0.9358 - val_loss: 0.2204 - val_accuracy: 0.9306
Epoch 5/10
78/78 [==============================] - 23s 296ms/step - loss: 0.1711 - accuracy: 0.9472 - val_loss: 0.2068 - val_accuracy: 0.9330
Epoch 6/10
78/78 [==============================] - 23s 296ms/step - loss: 0.1731 - accuracy: 0.9437 - val_loss: 0.2012 - val_accuracy: 0.9312
Epoch 7/10
78/78 [==============================] - 23s 296ms/step - loss: 0.1767 - accuracy: 0.9417 - val_loss: 0.2023 - val_accuracy: 0.9330

<h3>Bi-Directional</h3>

In [ ]:
from tensorflow.keras.layers import Bidirectional
## Creating model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense

model=Sequential()
model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix],input_length=165 ,trainable=False))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())



Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 165, 300)          5350800   
_________________________________________________________________
dropout_4 (Dropout)          (None, 165, 300)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               320800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 5,671,801
Trainable params: 321,001
Non-trainable params: 5,350,800
_________________________________________________________________
None


In [ ]:
model.fit(train_seq_x,train_y,validation_data=(valid_seq_x,valid_y),epochs=10,batch_size=64)

Epoch 1/10
78/78 [==============================] - 54s 652ms/step - loss: 0.5414 - accuracy: 0.7199 - val_loss: 0.4596 - val_accuracy: 0.8315
Epoch 2/10
78/78 [==============================] - 50s 642ms/step - loss: 0.4077 - accuracy: 0.8424 - val_loss: 0.3180 - val_accuracy: 0.8937
Epoch 3/10
78/78 [==============================] - 50s 642ms/step - loss: 0.2625 - accuracy: 0.9123 - val_loss: 0.2482 - val_accuracy: 0.9130
Epoch 4/10
78/78 [==============================] - 50s 644ms/step - loss: 0.2414 - accuracy: 0.9138 - val_loss: 0.2951 - val_accuracy: 0.8949
Epoch 5/10
78/78 [==============================] - 50s 643ms/step - loss: 0.2237 - accuracy: 0.9259 - val_loss: 0.2445 - val_accuracy: 0.9161
Epoch 6/10
78/78 [==============================] - 50s 644ms/step - loss: 0.1973 - accuracy: 0.9328 - val_loss: 0.2178 - val_accuracy: 0.9300
Epoch 7/10
78/78 [==============================] - 50s 642ms/step - loss: 0.1782 - accuracy: 0.9452 - val_loss: 0.2174 - val_accuracy: 0.9306